## This python notebook has examples of the Set-Coverage Problem and Partial Set-Coverage Problem
### load libraries

In [13]:
import geopandas as gpd
import pandas as pd
import os, sys, time
sys.path.append(r'/home/vagrant/repos/GOST_PublicGoods/GOSTNets/GOSTNets')
import GOSTnet as gn
import importlib
import networkx as nx
import osmnx as ox
from shapely.ops import unary_union
from shapely.wkt import loads
from shapely.geometry import LineString, MultiLineString, Point

### read back in processed graph

In [14]:
G = nx.read_gpickle('./sampleData/nouakchott/biggest_subg.pickle')

In [15]:
gn.example_edge(G)

(0, 23857, {'Wkt': 'LINESTRING (-15.8975893 18.0394637, -15.8976921 18.039127)', 'id': 45978, 'infra_type': 'residential', 'osm_id': '667426151', 'key': 'edge_45978', 'length': 38.82328422750167, 'Type': 'legitimate', 'time': 13.976382321900601, 'mode': 'drive'})


## load origins and destinations

In [16]:
origins = pd.read_csv('./sampleData/nouakchott/origins_test1.csv')
origins['geometry'] = list(zip(origins['Lon'],origins['Lat']))
origins['geometry'] = origins['geometry'].apply(Point)
origins_gdf = gpd.GeoDataFrame(origins, crs = {'init':'epsg:4326'}, geometry = 'geometry')
origins_gdf = gn.pandana_snap(G, origins_gdf, target_crs = 'epsg:32628', add_dist_to_node_col = True)
origins = list(origins_gdf.NN)
origins = list(set(origins))

In [17]:
destinations = pd.read_csv('./sampleData/nouakchott/destinations_test1.csv')
destinations['geometry'] = list(zip(destinations['Lon'],destinations['Lat']))
destinations['geometry'] = destinations['geometry'].apply(Point)
destinations_gdf = gpd.GeoDataFrame(destinations, crs = {'init':'epsg:4326'}, geometry = 'geometry')
destinations_gdf = gn.pandana_snap(G, destinations_gdf, target_crs = 'epsg:32628', add_dist_to_node_col = True)
#destinations_gdf.NN is the nearest node of the road network
destinations = list(destinations_gdf.NN)
destinations = list(set(destinations))

## calculate OD Matrix

In [18]:
%time OD = gn.calculate_OD(G, origins, destinations, fail_value = 9999999999999)

CPU times: user 2.2 s, sys: 2.87 ms, total: 2.21 s
Wall time: 2.28 s


In [19]:
OD_df = pd.DataFrame(OD, columns = destinations, index = origins)

In [20]:
OD_df[0:10]

,8994,7011,6724,4965,2440,1646,25486,33112,15739,4092
28291,3340.836168,1941.811792,2474.175924,1637.061354,1065.073847,2019.845992,2802.051017,2740.249661,2390.308863,2326.188681
15877,3719.561387,552.200337,1662.082592,1663.938809,2262.278821,3431.780571,2196.624544,2859.957422,2766.786612,1923.391222
33926,3774.268565,1292.827363,2257.561160,1196.118322,2316.985999,3486.487749,1490.138612,2222.673912,2819.961653,1017.160746
4101,2611.859344,2272.648285,2805.012416,1967.897847,575.529981,2532.306273,3132.887509,2695.357159,1661.431381,2657.025174
7178,2013.543292,1682.311487,2214.675618,1376.028912,1622.705275,2792.331903,2541.018574,1233.723334,1045.810811,2065.156239
15243,1622.658588,2097.960971,2630.325102,1791.678396,2038.354760,3207.981387,2956.668059,1927.828917,244.203522,2480.805723
14482,2911.656748,2009.776870,2542.141001,995.746620,2243.302682,3412.929310,1785.201938,366.536414,1938.141226,1309.339603
34965,2980.030795,3014.883868,3547.248000,2710.133431,1363.870746,2203.531169,3875.123093,3153.667045,2034.871046,3399.260757
11029,2550.781031,2089.009361,2621.373492,1244.331640,2029.403150,3199.029777,2313.401780,733.056700,1577.265509,1837.539444
22552,2459.436717,1533.001302,2065.365433,1228.250864,929.352851,2489.316195,2393.240526,1977.578287,1509.008754,1917.378191


In [21]:
OD_df.keys()

Int64Index([8994, 7011, 6724, 4965, 2440, 1646, 25486, 33112, 15739, 4092], dtype='int64')

In [23]:
OD_df.index

Int64Index([28291, 15877, 33926,  4101,  7178, 15243, 14482, 34965, 11029,
            22552, 30616, 33435, 32797, 19488, 27040, 17189, 23723,  2044,
            11310, 32820,  5048, 11322,  5307,  4798,  8638,  5702, 35016,
            15307, 26830, 16726, 27222, 12121, 27867, 32092,  5343, 26977,
             9699, 31973, 27750,  9703, 18544, 11633, 10098,  4851, 20596,
            18033, 31222, 36602, 34556, 32893],
           dtype='int64')

In [24]:
OD_df.loc[7178,6724]

2214.6756178992073

In [25]:
OD_df.keys()

Int64Index([8994, 7011, 6724, 4965, 2440, 1646, 25486, 33112, 15739, 4092], dtype='int64')

In [76]:
import importlib
importlib.reload(gn)

peartree version: 0.6.1 
networkx version: 2.2 
matplotlib version: 3.0.3 
osmnx version: 0.9 


<module 'GOSTnet' from '/home/vagrant/repos/GOST_PublicGoods/GOSTNets/GOSTNets/GOSTnet.py'>

## Set-Coverage Problem
### Objective: Determine the minimum number of facilities and their locations in order to cover all demands within a pre-specified maximum distance (or time) coverage

### inputs include a pre-specified maximum distance coverage of 3000 seconds

In [77]:
set_coverage_result = gn.optimize_set_coverage(OD_df,max_coverage = 2000)

number of origins
50
print totalCoveredOrigins
48
print percent coverage
96.0
print prob status
Optimal
print problem variables
[X_15739, X_1646, X_2440, X_25486, X_33112, X_4092, X_4965, X_6724, X_7011, X_8994, __dummy]
print objective value
<bound method LpAffineExpression.value of 0*__dummy + False>
print prob obj
0*__dummy


In [64]:
set_coverage_result

[]

## Partial Set-Coverage Problem
### Objective: Determine the minimum number of facilities and their locations in order to cover a given fraction of the population within a pre-specified maximum distance (or time) coverage

### we need to produce a series that has each origin and its respective population

In [110]:
origins_w_demands_series = pd.Series(origins_gdf.demand.values,index=origins_gdf.NN)

### inputs include covering 90 percent of the population, a pre-specified maximum distance coverage of 2000 seconds, and a  series of origins with their population

In [115]:
partial_set_coverage_result = gn.optimize_partial_set_coverage(OD_df, pop_coverage = .9, max_coverage = 2000, origins_pop_series = origins_w_demands_series, existing_facilities = None)

print min_coverage
12419780.4


In [112]:
partial_set_coverage_result

[15739, 1646, 2440, 25486, 33112, 4092, 4965, 6724, 7011, 8994]